In [37]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import h5py
import os

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [106]:
df = pd.read_csv('trainLabels.csv')
n = len(df)
synset = sorted(set(df['label']))
y = np.zeros(n)
x = np.zeros((n,10))

# for i, (fname, breed) in tqdm(df.iterrows(), total=n): 
for i in range(50000):
    i = i+1
    img = cv2.imread('CIFAR_train/%s.png' % i, 0)
    print(i)
    cv2.imshow('image',img)
    if(i == 50000):
        break
#     img_64 = cv2.resize(img, (8,8), interpolation=cv2.INTER_CUBIC)
    

1
2
3
4
5
6


error: /tmp/opencv-20171112-53254-axs83r/opencv-3.3.1/modules/highgui/src/window.cpp:331: error: (-215) size.width>0 && size.height>0 in function imshow


In [100]:
print(y)

[ 0.  0.  0. ...,  0.  0.  0.]


In [77]:
img = cv2.imread('CIFAR_train/%s.png' % 1, 0)
img_64 = cv2.resize(img, (8,8))
img = img_64.reshape((1, 64))
img.shape

(1, 64)

In [107]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [110]:
unpickle('cifar-10-batches-py/data_batch_1')

{b'batch_label': b'training batch 1 of 5',
 b'data': array([[ 59,  43,  50, ..., 140,  84,  72],
        [154, 126, 105, ..., 139, 142, 144],
        [255, 253, 253, ...,  83,  83,  84],
        ..., 
        [ 71,  60,  74, ...,  68,  69,  68],
        [250, 254, 211, ..., 215, 255, 254],
        [ 62,  61,  60, ..., 130, 130, 131]], dtype=uint8),
 b'filenames': [b'leptodactylus_pentadactylus_s_000004.png',
  b'camion_s_000148.png',
  b'tipper_truck_s_001250.png',
  b'american_elk_s_001521.png',
  b'station_wagon_s_000293.png',
  b'coupe_s_001735.png',
  b'cassowary_s_001300.png',
  b'cow_pony_s_001168.png',
  b'sea_boat_s_001584.png',
  b'tabby_s_001355.png',
  b'muntjac_s_001000.png',
  b'arabian_s_001354.png',
  b'quarter_horse_s_000672.png',
  b'passerine_s_000343.png',
  b'camion_s_001895.png',
  b'trailer_truck_s_000335.png',
  b'dumper_s_000821.png',
  b'alley_cat_s_000200.png',
  b'accentor_s_000677.png',
  b'frog_s_001671.png',
  b'capreolus_capreolus_s_000051.png',
  b'tomca

In [114]:
import numpy as np  
import os  
import pickle  
import glob  
import matplotlib.pyplot as plt  
  
data_dir = "Downloads"  
data_dir_cifar10 = os.path.join('', "cifar-10-batches-py")  
  
class_names_cifar10 = np.load(os.path.join(data_dir_cifar10, "batches.meta"))  

In [125]:
def _load_batch_cifar10(filename, dtype='float64'):  
    """ 
    load a batch in the CIFAR-10 format 
    """  
    path = os.path.join(data_dir_cifar10, filename)  
    batch = np.load(path)  
    data = batch['data'] / 255.0 # scale between [0, 1]  
    labels = one_hot(batch['labels'], n=10) # convert labels to one-hot representation  
    return data.astype(dtype), labels.astype(dtype)  
    
def _grayscale(a):  
    print(a.reshape(a.shape[0], 3, 32, 32).mean(1).reshape(a.shape[0], -1)) 
    return a.reshape(a.shape[0], 3, 32, 32).mean(1).reshape(a.shape[0], -1)  
  
def cifar10(dtype='float64', grayscale=True):  
    # train  
    x_train = []  
    t_train = []  
    for k in range(4):  
        x = _load_batch_cifar10('data_batch_%d' % (k + 1), dtype=dtype)  
        x_train.append(x)  
#         t_train.append(t)  
  
    x_train = np.concatenate(x_train, axis=0)  
#     t_train = np.concatenate(t_train, axis=0)  
  
    # test  
    x_test = _load_batch_cifar10('test_batch', dtype=dtype)   
#     if grayscale:  
#         x_train = _grayscale(x_train)  
#         x_test = _grayscale(x_test)  
  
    return x_train, t_train, x_test, t_test

In [139]:
data_dir_cifar10 = os.path.join('', "cifar-10-batches-py")  
path = os.path.join(data_dir_cifar10, 'data_batch_1')  
with open(path, 'rb') as fo:
    batch = pickle.load(fo, encoding = 'bytes')  
batch[b'track']
# scale between [0, 1]  
# labels = one_hot(batch['labels'], n=10) # convert labels to one-hot representation  
# return data.astype(dtype), labels.astype(dtype)

KeyError: b'track'

In [166]:
########################################################################
#
# Functions for downloading the CIFAR-10 data-set from the internet
# and loading it into memory.
#
# Implemented in Python 3.5
#
# Usage:
# 1) Set the variable data_path with the desired storage path.
# 2) Call maybe_download_and_extract() to download the data-set
#    if it is not already located in the given data_path.
# 3) Call load_class_names() to get an array of the class-names.
# 4) Call load_training_data() and load_test_data() to get
#    the images, class-numbers and one-hot encoded class-labels
#    for the training-set and test-set.
# 5) Use the returned data in your own program.
#
# Format:
# The images for the training- and test-sets are returned as 4-dim numpy
# arrays each with the shape: [image_number, height, width, channel]
# where the individual pixels are floats between 0.0 and 1.0.
#
########################################################################
#
# This file is part of the TensorFlow Tutorials available at:
#
# https://github.com/Hvass-Labs/TensorFlow-Tutorials
#
# Published under the MIT License. See the file LICENSE for details.
#
# Copyright 2016 by Magnus Erik Hvass Pedersen
#
########################################################################

import numpy as np
import pickle
import os

########################################################################

# Directory where you want to download and save the data-set.
# Set this before you start calling any of the functions below.
data_path = ""

# URL for the data-set on the internet.

########################################################################
# Various constants for the size of the images.
# Use these constants in your own program.

# Width and height of each image.
img_size = 32

# Number of channels in each image, 3 channels: Red, Green, Blue.
num_channels = 3

# Length of an image when flattened to a 1-dim array.
img_size_flat = img_size * img_size * num_channels

# Number of classes.
num_classes = 10

########################################################################
# Various constants used to allocate arrays of the correct size.

# Number of files for the training-set.
_num_files_train = 5

# Number of images for each batch-file in the training-set.
_images_per_file = 10000

# Total number of images in the training-set.
# This is used to pre-allocate arrays for efficiency.
_num_images_train = _num_files_train * _images_per_file

########################################################################
# Private functions for downloading, unpacking and loading data-files.


def _get_file_path(filename=""):
    """
    Return the full path of a data-file for the data-set.
    If filename=="" then return the directory of the files.
    """

    return os.path.join(data_path, "cifar-10-batches-py/", filename)


def _unpickle(filename):
    """
    Unpickle the given file and return the data.
    Note that the appropriate dir-name is prepended the filename.
    """

    # Create full path for the file.
    file_path = _get_file_path(filename)

    print("Loading data: " + file_path)

    with open(file_path, mode='rb') as file:
        # In Python 3.X it is important to set the encoding,
        # otherwise an exception is raised here.
        data = pickle.load(file, encoding='bytes')

    return data


def _convert_images(raw):
    """
    Convert images from the CIFAR-10 format and
    return a 4-dim array with shape: [image_number, height, width, channel]
    where the pixels are floats between 0.0 and 1.0.
    """

    # Convert the raw images from the data-files to floating-points.
    raw_float = np.array(raw, dtype=float) / 255.0

    # Reshape the array to 4-dimensions.
    images = raw_float.reshape([-1, num_channels, img_size, img_size])

    # Reorder the indices of the array.
    images = images.transpose([0, 2, 3, 1])

    return images


def _load_data(filename):
    """
    Load a pickled data-file from the CIFAR-10 data-set
    and return the converted images (see above) and the class-number
    for each image.
    """

    # Load the pickled data-file.
    data = _unpickle(filename)

    # Get the raw images.
    raw_images = data[b'data']

    # Get the class-numbers for each image. Convert to numpy-array.
    cls = np.array(data[b'labels'])

    # Convert the images.
    images = _convert_images(raw_images)

    return images, cls


########################################################################
# Public functions that you may call to download the data-set from
# the internet and load the data into memory.



def load_class_names():
    """
    Load the names for the classes in the CIFAR-10 data-set.
    Returns a list with the names. Example: names[3] is the name
    associated with class-number 3.
    """

    # Load the class-names from the pickled file.
    raw = _unpickle(filename="batches.meta")[b'label_names']

    # Convert from binary strings.
    names = [x.decode('utf-8') for x in raw]

    return names


def load_training_data():
    """
    Load all the training-data for the CIFAR-10 data-set.
    The data-set is split into 5 data-files which are merged here.
    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    # Pre-allocate the arrays for the images and class-numbers for efficiency.
    images = np.zeros(shape=[_num_images_train, img_size, img_size, num_channels], dtype=float)
    cls = np.zeros(shape=[_num_images_train], dtype=int)

    # Begin-index for the current batch.
    begin = 0

    # For each data-file.
    for i in range(_num_files_train):
        # Load the images and class-numbers from the data-file.
        images_batch, cls_batch = _load_data(filename="data_batch_" + str(i + 1))

        # Number of images in this batch.
        num_images = len(images_batch)

        # End-index for the current batch.
        end = begin + num_images

        # Store the images into the array.
        images[begin:end, :] = images_batch

        # Store the class-numbers into the array.
        cls[begin:end] = cls_batch

        # The begin-index for the next batch is the current end-index.
        begin = end

        print(images.shape)
    return images, cls


def load_test_data():
    """
    Load all the test-data for the CIFAR-10 data-set.
    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    images, cls = _load_data(filename="test_batch")

    return images, cls

def save_data():
    np.savetxt('CIFAR.txt',load_training_data())

In [167]:
save_data()

Loading data: cifar-10-batches-py/data_batch_1
(50000, 32, 32, 3)
Loading data: cifar-10-batches-py/data_batch_2
(50000, 32, 32, 3)
Loading data: cifar-10-batches-py/data_batch_3
(50000, 32, 32, 3)
Loading data: cifar-10-batches-py/data_batch_4
(50000, 32, 32, 3)
Loading data: cifar-10-batches-py/data_batch_5
(50000, 32, 32, 3)


ValueError: could not broadcast input array from shape (50000,32,32,3) into shape (50000)

In [168]:
''''' 
    首先把官网的python版本数据下载保存到本地 
'''  
file1 = 'cifar-10-batches-py/data_batch_1'  
file2 = 'cifar-10-batches-py/batches.meta'  
file3 = 'cifar-10-batches-py/test_batch'  
  
  
def unpickle(file):  # 该函数将cifar10提供的文件读取到python的数据结构(字典)中  
    import pickle  
    fo = open(file, 'rb')  
    dict = pickle.load(fo,encoding='iso-8859-1')  
    fo.close()  
    return dict  
  
dict_train_batch1 = unpickle(file1) # 将data_batch文件读入到数据结构(字典)中  
  
print(dict_train_batch1) # 每个batch是一个字典  
print(dict_train_batch1.keys())  # 字典里有4组键值对  
  
''''' 
    trainSet字典里有4组键值对 
    1，batch_label ：表明batch的位置,没什么用 
    2，data ：32*32图片的数值化数组，是一个10000*3072的numpy二维数组, 
              每一行代表一张图片，一行分3段(红绿蓝色道)，每段1024个元素。 
    3，labels ：data每一行对应的标签（数字0-9），是个一维数组，10000个元素 
    4，filenames ： data每一行对应的文件名，同是一个一维数组，10000个元素 
'''  
data_train_batch1 = dict_train_batch1.get('data') # 字典中取data  
print(data_train_batch1)  
  
labels = dict_train_batch1.get('labels') # 字典中取labels  
print(labels)  
  
filenames = dict_train_batch1.get('filenames') # 字典中取filenames  
print(filenames)  
  
  
print('--------------我是分割线---------------------------------')  
  
  
dict_test_batch = unpickle(file3)  
print(dict_test_batch)  
print(dict_test_batch.keys())  
''''' 
    跟trainSet中一样，testSet中有相同的4组键值对 
    说明同上 
'''  
  
print('--------------我是分割线---------------------------------')  
  
  
''''' 
    batches.meta是一个字典，其中包含了一个列表，列表中是10种分类的具体名称 
    一般这个文件用不到 
'''  
dict_meta_batch = unpickle(file2)  
print(dict_meta_batch)

{'batch_label': 'training batch 1 of 5', 'labels': [6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3, 6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4, 0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5, 6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2, 1, 1, 2, 1, 5, 9, 9, 0, 8, 4, 1, 1, 6, 3, 3, 

In [169]:
#encoding:utf-8
#author：guohuifeng
#email：309884616@qq.com
import numpy as np

ImportError: cannot import name 'imsave'

In [219]:
def unpickle(file):
    import pickle
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='bytes')
    fo.close()
    return dict

In [224]:
import cv2
# 生成训练集图片，如果需要png格式，只需要改图片后缀名即可。
x = np.zeros((50000,3072))
for j in range(1, 6):
    dataName = "cifar-10-batches-py/data_batch_" + str(j)  # 读取当前目录下的data_batch12345文件，dataName其实也是data_batch文件的路径，本文和脚本文件在同一目录下。
    Xtr = unpickle(dataName)
    print(dataName + " is loading...")

    for i in range(0, 10000):
        img = np.reshape(Xtr[b'data'][i], (3, 32, 32))  # Xtr['data']为图片二进制数据
        img = img.transpose(1, 2, 0)  # 读取image
        picName = 'CIFAR10_train/' + str(Xtr[b'labels'][i]) + '_' + str(i + (j - 1)*10000) + '.jpg'  
        # Xtr['labels']为图片的标签，值范围0-9，本文中，train文件夹需要存在，并与脚本文件在同一目录下。
        cv2.imwrite(picName, img)
    print(dataName + " loaded.")

cifar-10-batches-py/data_batch_1 is loading...
cifar-10-batches-py/data_batch_1 loaded.
cifar-10-batches-py/data_batch_2 is loading...
cifar-10-batches-py/data_batch_2 loaded.
cifar-10-batches-py/data_batch_3 is loading...
cifar-10-batches-py/data_batch_3 loaded.
cifar-10-batches-py/data_batch_4 is loading...
cifar-10-batches-py/data_batch_4 loaded.
cifar-10-batches-py/data_batch_5 is loading...
cifar-10-batches-py/data_batch_5 loaded.


In [223]:
from scipy.misc import imsave

ImportError: cannot import name 'imsave'

In [222]:
file_CIFAR10 = 'CIFAR10_matrixs.txt'
with open(file_CIFAR10, 'wb') as h:
    np.savetxt(h, x)

KeyboardInterrupt: 